In [1]:
import QuantLib as ql
import openpyxl as xl

In [2]:
def DatetimeToDate(ADatetime):
    ADate = ql.Date(ADatetime.day, ADatetime.month, ADatetime.year)
    return ADate

In [3]:
PstBook = xl.load_workbook('Pst_FXOption.xlsx', data_only=True)
MktSheet = PstBook.worksheets[0]
CommCCY = 'USD'
TermCCY = 'TWD'
Type = 'OPT'
MTMDate = MktSheet.cell(1, 3).value
FXSpot = MktSheet.cell(3, 6).value

In [4]:
settings = ql.Settings.instance()
evDate = DatetimeToDate(MTMDate)
settings.setEvaluationDate(evDate)
Cal = ql.NullCalendar()
CommDC = ql.Actual360()
TermDC = ql.Actual365Fixed()
DC365 = ql.Actual365Fixed()
BDC = ql.Unadjusted
settlementDays = 0

In [5]:
CommZeroSP = MktSheet.cell(22, 6).value
CommZero3M = MktSheet.cell(23, 6).value
CommZero6M = MktSheet.cell(24, 6).value
CommZero1Y = MktSheet.cell(25, 6).value
print(CommZeroSP)

0.001459727696536763


In [6]:
DateSP = evDate
Date3M = Cal.advance(evDate, 3, ql.Months, BDC, False)
Date6M = Cal.advance(evDate, 6, ql.Months, BDC, False)
Date1Y = Cal.advance(evDate, 1, ql.Years, BDC, False)
print(DateSP)

June 30th, 2021


In [7]:
TermZeroSP = MktSheet.cell(22, 12).value
TermZero3M = MktSheet.cell(23, 12).value
TermZero6M = MktSheet.cell(24, 12).value
TermZero1Y = MktSheet.cell(25, 12).value
print(TermZeroSP)

-0.00392904297456824


In [8]:
ListDate = list([DateSP, Date3M, Date6M, Date1Y])
ListCommRate = list([CommZeroSP, CommZero3M, CommZero6M, CommZero1Y])
ListTermRate = list([TermZeroSP, TermZero3M, TermZero6M, TermZero1Y])
ListDate
ListCommRate
ListTermRate

[-0.00392904297456824,
 -0.003929042973135284,
 -0.0006520395000015291,
 0.001329938573603508]

In [9]:
VecDate = ql.DateVector(ListDate)
VecCommRate = ql.DoubleVector(ListCommRate)
VecTermRate = ql.DoubleVector(ListTermRate)

In [10]:
TSCommZero = ql.ZeroCurve(VecDate, VecCommRate, CommDC, Cal, ql.Linear(), 
                          ql.Continuous, ql.Annual)
h_TSCommZero = ql.YieldTermStructureHandle(TSCommZero)
TSTermZero = ql.ZeroCurve(VecDate, VecTermRate, TermDC, Cal, ql.Linear(), 
                          ql.Continuous, ql.Annual)
h_TSTermZero = ql.YieldTermStructureHandle(TSTermZero)

In [11]:
ListVolDate = list([Date6M, Date1Y])
ListStrike = list()
for i in range(9):
    ListStrike.append(MktSheet.cell(31+i, 15).value)
ListStrike

[26.75, 27, 27.25, 27.5, 27.75, 28, 28.25, 28.5, 28.75]

In [12]:
NoVolCurve = len(ListVolDate)
print(NoVolCurve)
NoStrike = len(ListStrike)
print(NoStrike)

2
9


In [13]:
VecVolDate = ql.DateVector(ListVolDate)
VecStrike = ql.DoubleVector(ListStrike)
M_Vol = ql.Matrix(9, 2, 0.0)

In [14]:
for i in range(NoStrike):
    for j in range(NoVolCurve):
        M_Vol[i][j] = MktSheet.cell(31+i, 16+j).value
print(M_Vol)   

| 0.215209 0.113231 |
| 0.138996 0.0627988 |
| 0.0627825 0.0579179 |
| 0.0535264 0.0570613 |
| 0.0524342 0.0562047 |
| 0.0527047 0.0561806 |
| 0.0542796 0.0573503 |
| 0.0558546 0.0585201 |
| 0.0868235 0.0596899 |



In [15]:
ListM_Vol = list()
ListM_Vol.append(M_Vol)
for k in range(NoVolCurve):
    NewM_Vol = ql.Matrix(9, 2, 0.0)    
    for i in range(NoStrike): 
        for j in range(NoVolCurve):
            if(j==k):
                NewM_Vol[i][k] = M_Vol[i][k] + 0.0001
            else:
                NewM_Vol[i][j] = M_Vol[i][j]
    ListM_Vol.append(NewM_Vol)

In [16]:
print(len(ListM_Vol))
print(ListM_Vol[0])
print(ListM_Vol[1])
print(ListM_Vol[2])

3
| 0.215209 0.113231 |
| 0.138996 0.0627988 |
| 0.0627825 0.0579179 |
| 0.0535264 0.0570613 |
| 0.0524342 0.0562047 |
| 0.0527047 0.0561806 |
| 0.0542796 0.0573503 |
| 0.0558546 0.0585201 |
| 0.0868235 0.0596899 |

| 0.215309 0.113231 |
| 0.139096 0.0627988 |
| 0.0628825 0.0579179 |
| 0.0536264 0.0570613 |
| 0.0525342 0.0562047 |
| 0.0528047 0.0561806 |
| 0.0543796 0.0573503 |
| 0.0559546 0.0585201 |
| 0.0869235 0.0596899 |

| 0.215209 0.113331 |
| 0.138996 0.0628988 |
| 0.0627825 0.0580179 |
| 0.0535264 0.0571613 |
| 0.0524342 0.0563047 |
| 0.0527047 0.0562806 |
| 0.0542796 0.0574503 |
| 0.0558546 0.0586201 |
| 0.0868235 0.0597899 |



In [17]:
Listh_VolSurface = list()
for k in range(NoVolCurve+1):
    NewVolSurface = ql.BlackVarianceSurface(evDate, Cal, VecVolDate, VecStrike,
                                            ListM_Vol[k], DC365)
    Newh_VolSurface = ql.BlackVolTermStructureHandle(NewVolSurface)
    Listh_VolSurface.append(Newh_VolSurface)

In [18]:
print(len(Listh_VolSurface))

3


In [19]:
aVol = Listh_VolSurface[0].blackVol(0.75, 27.891)
print(aVol)

0.05500941759027123


In [20]:
OPTSheet = PstBook['Option']

In [21]:
start_row = 2
end_row = 4
start_col = 0
end_col = 11
recordcount = end_col - start_col + 1
ListOPT = list()
for i in range(start_row, end_row + 1):
    row = [cell.value for cell in OPTSheet[i][start_col:end_col+1]]
    ListOPT.append(row)
    print(row)

['OPT001', datetime.datetime(2021, 5, 19, 0, 0), datetime.datetime(2021, 5, 21, 0, 0), datetime.datetime(2021, 9, 21, 0, 0), 'Long', 'Call', 'VLN', 'USDTWD', 'USD', 'TWD', 28, 1000000]
['OPT002', datetime.datetime(2021, 6, 13, 0, 0), datetime.datetime(2021, 6, 15, 0, 0), datetime.datetime(2021, 12, 15, 0, 0), 'Short', 'Call', 'VLN', 'USDTWD', 'USD', 'TWD', 27.8, 1000000]
['OPT003', datetime.datetime(2021, 6, 21, 0, 0), datetime.datetime(2021, 6, 23, 0, 0), datetime.datetime(2022, 5, 23, 0, 0), 'Long', 'Put', 'VLN', 'USDTWD', 'USD', 'TWD', 28.1, 1000000]


In [22]:
OPTBook = list()
for i in range(len(ListOPT)):
    row = ListOPT[i]
    record = list()
    record.append(row[0])
    record.append(DatetimeToDate(row[1]))
    record.append(DatetimeToDate(row[2]))
    record.append(DatetimeToDate(row[3]))
    record.append(row[4])
    record.append(row[5])
    record.append(row[6])
    record.append(row[7])
    record.append(row[8])
    record.append(row[9])
    record.append(row[10])
    record.append(row[11])
    OPTBook.append(record)

In [23]:
OPTBook

[['OPT001',
  Date(19,5,2021),
  Date(21,5,2021),
  Date(21,9,2021),
  'Long',
  'Call',
  'VLN',
  'USDTWD',
  'USD',
  'TWD',
  28,
  1000000],
 ['OPT002',
  Date(13,6,2021),
  Date(15,6,2021),
  Date(15,12,2021),
  'Short',
  'Call',
  'VLN',
  'USDTWD',
  'USD',
  'TWD',
  27.8,
  1000000],
 ['OPT003',
  Date(21,6,2021),
  Date(23,6,2021),
  Date(23,5,2022),
  'Long',
  'Put',
  'VLN',
  'USDTWD',
  'USD',
  'TWD',
  28.1,
  1000000]]

In [24]:
# Delta & Curvature
SenSheet = PstBook['FXR']
for i in range(len(OPTBook)):
    record = OPTBook[i]
    tradeDate = record[1]
    maturityDate = record[3]
    position = record[4]
    OptClass = record[5]
    OptType = record[6]
    VolClass = record[7]
    CommCCY = record[8]
    TermCCY = record[9]
    Strike = record[10]
    CommAmt = record[11]    
    EuExer = ql.EuropeanExercise(maturityDate)
    if (OptClass == 'Call'):
        VLPayoff = ql.PlainVanillaPayoff(ql.Option.Call, Strike)
    else:
        VLPayoff = ql.PlainVanillaPayoff(ql.Option.Put, Strike)
    
    anEuroOption = ql.EuropeanOption(VLPayoff, EuExer)
    Q_Spot = ql.SimpleQuote(FXSpot)
    hQ_Spot = ql.QuoteHandle(Q_Spot)
    h_VolSurface = Listh_VolSurface[0]
    GBSProcess = ql.GeneralizedBlackScholesProcess(hQ_Spot, h_TSCommZero, h_TSTermZero, h_VolSurface)
    AEEngine = ql.AnalyticEuropeanEngine(GBSProcess)
    anEuroOption.setPricingEngine(AEEngine)
    if (position == 'Long'):
        Sign = 1.0
    else:
        Sign = -1.0
    
    Value = Sign * CommAmt * anEuroOption.NPV()
    print(Value)
        
    FXSpot_1 = FXSpot * 1.01
    Q_Spot_1 = ql.SimpleQuote(FXSpot_1)
    hQ_Spot_1 = ql.QuoteHandle(Q_Spot_1)
    GBSProcess_1 = ql.GeneralizedBlackScholesProcess(hQ_Spot_1, h_TSCommZero, h_TSTermZero, h_VolSurface)
    AEEngine_1 = ql.AnalyticEuropeanEngine(GBSProcess_1)
    anEuroOption.setPricingEngine(AEEngine_1)
    Value_1 = Sign * CommAmt * anEuroOption.NPV()    
    Delta = (Value_1 - Value) * 100.0
    print(Value_1)
                    
    FXSpot_Up = FXSpot * 1.15
    Q_Spot_Up = ql.SimpleQuote(FXSpot_Up)
    hQ_Spot_Up = ql.QuoteHandle(Q_Spot_Up)
    GBSProcess_Up = ql.GeneralizedBlackScholesProcess(hQ_Spot_Up, h_TSCommZero, h_TSTermZero, h_VolSurface)
    AEEngine_Up = ql.AnalyticEuropeanEngine(GBSProcess_Up)
    anEuroOption.setPricingEngine(AEEngine_Up)
    Value_Up = Sign * CommAmt * anEuroOption.NPV()    
    print(Value_Up)
    
    FXSpot_Dn = FXSpot * 0.85
    Q_Spot_Dn = ql.SimpleQuote(FXSpot_Dn)
    hQ_Spot_Dn = ql.QuoteHandle(Q_Spot_Dn)
    GBSProcess_Dn = ql.GeneralizedBlackScholesProcess(hQ_Spot_Dn, h_TSCommZero, h_TSTermZero, h_VolSurface)
    AEEngine_Dn = ql.AnalyticEuropeanEngine(GBSProcess_Dn)
    anEuroOption.setPricingEngine(AEEngine_Dn)
    Value_Dn = Sign * CommAmt * anEuroOption.NPV()     
    print(Value_Dn)
    
    Curv_Up = (-((Value_Up - Value) - 0.15 * Delta))
    Curv_Dn = (-((Value_Dn - Value) + 0.15 * Delta))            
                
    SenSheet.cell(i+5, 1).value = record[0]
    SenSheet.cell(i+5, 2).value = Value
    SenSheet.cell(i+5, 3).value = CommCCY
    SenSheet.cell(i+5, 4).value = Delta
    SenSheet.cell(i+5, 6).value = Curv_Up
    SenSheet.cell(i+5, 8).value = Curv_Dn
    # print(Delta)
    # print(Curv_Up)
    # print(Curv_Dn)    
PstBook.save('Pst_FXOption.xlsx')

214380.82208287125
354531.1593567521
4038829.2719921963
1.1871898143033044e-06
-421230.21349021135
-583512.5392568522
-4230322.303311605
-0.5953726738888246
727194.1754706659
582777.7132123939
3662.3825932819095
4424198.570729044


In [25]:
# Vega
SenSheet = PstBook['FXR']
for i in range(len(OPTBook)):
    ListValue = list()
    ListVega = list()
    ListSensitivity = list()
    record = OPTBook[i]
    tradeDate = record[1]
    maturityDate = record[3]
    position = record[4]
    OptClass = record[5]
    OptType = record[6]
    VolClass = record[7]
    CommCCY = record[8]
    TermCCY = record[9]
    Strike = record[10]
    CommAmt = record[11]    
    EuExer = ql.EuropeanExercise(maturityDate)
    if (OptClass == 'Call'):
        VLPayoff = ql.PlainVanillaPayoff(ql.Option.Call, Strike)
    else:
        VLPayoff = ql.PlainVanillaPayoff(ql.Option.Put, Strike)
    
    anEuroOption = ql.EuropeanOption(VLPayoff, EuExer)
    Q_Spot = ql.SimpleQuote(FXSpot)
    hQ_Spot = ql.QuoteHandle(Q_Spot)    
    h_VolSurface = Listh_VolSurface[0]
    GBSProcess = ql.GeneralizedBlackScholesProcess(hQ_Spot, h_TSCommZero, h_TSTermZero, h_VolSurface)
    AEEngine = ql.AnalyticEuropeanEngine(GBSProcess)
    anEuroOption.setPricingEngine(AEEngine)
    if (position == 'Long'):
        Sign = 1.0
    else:
        Sign = -1.0
    
    npv = anEuroOption.NPV()
    Value = Sign * CommAmt * npv
    print(Value)
    impVol = anEuroOption.impliedVolatility(npv, GBSProcess)
    print(impVol)
    ListValue.append(Value)
    ListVega.append(0.0)
    ListSensitivity.append(0.0)
        
    for k in range(1, NoVolCurve+1):
        h_VolSurface = Listh_VolSurface[k]
        GBSProcess = ql.GeneralizedBlackScholesProcess(hQ_Spot, h_TSCommZero, h_TSTermZero, h_VolSurface)
        AEEngine = ql.AnalyticEuropeanEngine(GBSProcess)
        anEuroOption.setPricingEngine(AEEngine)
        npv = anEuroOption.NPV()
        Value_1 = Sign * CommAmt * npv
        #print(Value_1)
        Vega = (Value_1 - Value) * 10000
        #print(Vega)        
        impVol = anEuroOption.impliedVolatility(npv, GBSProcess)
        #print(impVol)
        Sensitivity = Vega * impVol                    
        #print(Sensitivity)
        ListValue.append(Sensitivity)
        ListVega.append(Vega)
        ListSensitivity.append(Sensitivity)
                
    SenSheet.cell(i+5, 10).value = 'USDTWD'
    for j in range(1, NoVolCurve+1):
        SenSheet.cell(i+5, j+10).value = ListSensitivity[j]
        
PstBook.save('Pst_FXOption.xlsx')
PstBook.close()

214380.82208287125
0.052686797387009446
-421230.21349021135
0.05249415449288447
727194.1754706659
0.05627365875743092
